## Notes
<list>
<li>efective utilization and tardiness
<li>reward function
<li>Starting PointÖ
<li>2 recipes
<li>3 machines
<li>deadlines
</list>

### Questions:
<list>
<li> Is the queue of jobs part of the environment?
<li> Does the environment know which task of which job it is performing?
<li> 
<li> 
<li> 
<li> 
</list>


In [23]:
class Machine:
    machine_type = "A" #A,B,C,D...
    knownRecipes = [] #R1,R2,R3... 
    trayCapacity = 10000
    status = 0    # 0-Free, 1-Busy - I am asuming you cannot open a machine that is workingt to add more trays
    activeRecipe = []
    activeJobs = []
    
    def __init__(self, m_type, k_recipes, cap):
        self.machine_type = m_type
        self.knownRecipes=k_recipes
        self.trayCapacity = cap
        self.status = 0 #Free by default
    
    def getKnownRecipes(self):
        return self.knownRecipes
    
    def getMachineType(self):
        return self.machine_type
    
    def getTrayCapacity(self):
        return self.trayCapacity
    
    def getStatus(self):
        return self.status
    
    def getTextStatus(self):
        if self.status == 0:
            return "Free"
        else:
            return "Busy"
    
    def updateKnownRecipes(self,new_recipes):
        self.knownRecipes=new_recipes
    
    def updateTrayCapacity(self,new_capacity):
        self.trayCapacity=new_capacity
    
    def assignJobs(self, jobs,recipe):
        assigned = self.setActiveRecipe(recipe)
        if assigned:
            self.activeJobs.append(jobs)
            for j in jobs:
                j.updateProgress(recipe)
            self.status = 1
            return True
        else:
            return False
    
    def setActiveRecipe(self, recipe):
        if recipe in knownRecipes:
            self.activeRecipe = recipe
            return True
        else:
            return False
            
    def __str__(self):
        return f'Type: {self.machine_type} \nKnown Recipes: {self.knownRecipes} \nTray Capacity: {self.trayCapacity} \nStatus: {self.getTextStatus()}\nWorking on recipe: {self.activeRecipe} for the following Jobs: {self.activeJobs}'
    
class Job:
    jobId = "ID-00000" #String for now, can be numeric
    recipes = [] #R1,R2,R3...
    quantity = 0
    deadline = "3000/01/01" #YYYY/MM/DD
    status = 0
    priority = 1
    r_pending =[]
    r_inProgress = []
    r_completed=[]
    
    def __init__(self, jobId, recipes, quantity,deadline, priority):
        self.jobId = jobId
        self.recipes=recipes
        self.r_pending = recipes.copy()
        self.quantity = quantity
        self.deadline = deadline
        self.priority = priority # 1-Normal, 2-Medium, 3-High
        self.status = 0   # 0-New, 1- In Progress, 2- Completed, 3- Cancelled?, 4-Error? (For internal use)
    
    def getRecipes(self):
        return self.recipes
    
    def getjobId(self):
        return self.jobId
    
    def getQuantity(self):
        return self.quantity
    
    def getDeadline(self):
        return self.deadline
    
    def getStatus(self):
        return self.status #might come in handy in the future
    
    def getPriority(self):
        return self.priority
    
    def getTextPriority(self):
        ref_table = {1:"Normal", 2:'Medium',3:"High"}
        try:
            return  ref_table[self.priority]
        except:
            return "Not Defined"
    
    def getTextStatus(self):
        ref_table = {0:"New", 1:'In Progress',2:"Completed",3:"Cancelled",4:"Error"}
        try:
            return  ref_table[self.status]
        except:
            return "Not Defined"
        
    def updateDeadline(self,new_deadline):
        self.deadline=new_deadline
    
    def updatePriority(self,new_priority):
        self.priority = new_priority
    
    def updateStatus(self,new_status):
        self.status = new_status
    #I didnt add an update recipes because it would add complexity in the future if someone can alter the recipes
    
    def recipeInProgress(self,recipe):
        self.r_pending.remove(recipe)
        self.r_inProgress.append(recipe)
        self.status = 1
    
    def recipeCompleted(self, recipe):
        self.r_completed.append(recipe)
        self.r_inProgress.remove(recipe)
        if self.r_pending == [] and self.r_inProgress == []:
            self.status = 2
            
    def __str__(self):
        return f'Job ID: {self.jobId} \nRecipes: {self.recipes} \nQuantity: {self.quantity} \nDeadline: {self.deadline} \nPriority: {self.getTextPriority()} \nStatus: {self.getTextStatus()} \nIn Progress: {self.r_inProgress} \nCompleted: {100*len(self.r_completed)/len(self.recipes)} %'

In [24]:
m1 = Machine(m_type="A", k_recipes=[1,2], cap=10000)
m2 = Machine(m_type="A", k_recipes=[3,2], cap=10000)
m3 = Machine(m_type="A", k_recipes=[2], cap=10000)
print(m1)
print(m2)
print(m3)
print()
j1= Job(jobId="ID-00000", recipes=["A1","A2"], quantity=3,deadline="2024/01/04", priority="1")
j2= Job(jobId="ID-00001", recipes=["A2","A3"], quantity=10,deadline="2024/02/04", priority="2")
j3= Job(jobId="ID-00002", recipes=["A3"], quantity=5,deadline="2023/12/04", priority="3")
j1.recipeInProgress("A1")
j1.recipeCompleted("A1")
print(j1)
print("/--------/")
print(j2)
print("/--------/")
print(j3)

Type: A 
Known Recipes: [1, 2] 
Tray Capacity: 10000 
Status: Free
Working on recipe: [] for the following Jobs: []
Type: A 
Known Recipes: [3, 2] 
Tray Capacity: 10000 
Status: Free
Working on recipe: [] for the following Jobs: []
Type: A 
Known Recipes: [2] 
Tray Capacity: 10000 
Status: Free
Working on recipe: [] for the following Jobs: []

Job ID: ID-00000 
Recipes: ['A1', 'A2'] 
Quantity: 3 
Deadline: 2024/01/04 
Priority: Not Defined 
Status: In Progress 
In Progress: [] 
Completed: 50.0 %
/--------/
Job ID: ID-00001 
Recipes: ['A2', 'A3'] 
Quantity: 10 
Deadline: 2024/02/04 
Priority: Not Defined 
Status: New 
In Progress: [] 
Completed: 50.0 %
/--------/
Job ID: ID-00002 
Recipes: ['A3'] 
Quantity: 5 
Deadline: 2023/12/04 
Priority: Not Defined 
Status: New 
In Progress: [] 
Completed: 100.0 %


In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import matplotlib.pyplot as plt

#Base source: https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html


class FactoryEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["vector"]}

    def __init__(self, machines):
        super(FactoryEnv,self).__init__(machines)
        # Define action and observation space
        # They must be gym.spaces objects
        self.machines = machines
        self.action_space = spaces.Discrete(len(self.machines)) # possible machines to allocate jobs
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, len(self.machines)), dtype=int)
        #PARAMETERS:
            # low (Union[SupportsFloat, np.ndarray]) – Lower bounds of the intervals.
            # high (Union[SupportsFloat, np.ndarray]) – Upper bounds of the intervals.
            # shape (Optional[Sequence[int]]) – The shape is inferred from the shape of low or high np.ndarray`s with `low and high scalars defaulting to a shape of (1,)
            # dtype – The dtype of the elements of the space. If this is an integer type, the Box is essentially a discrete space.
            # seed – Optionally, you can use this argument to seed the RNG that is used to sample from the space.


    def step(self, action, jobs):
        #Return values definition
        reward = 0
        info=""
        terminated = False
        truncated = False
        
        if action not in range(len(self.machines)):
            truncated = True
            info = "Invalid Machine"
        
        self.machines[action].assignJob(jobs)
        
        #still have pending the time management of the processes
        
        if self.elapsed_time > self.max_steps:
            terminated=True

        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        self.board = np.zeros((6, 7), dtype=int)
        return self.board
        #return observation, info

    def render(self):
        for m in machines:
            print(m)
            print("/-------------------------/")

    def close(self):
        self.reset()